In [1]:
# Plot cross-shelf transects of temperature evolution on Amundsen sea shelf

In [1]:
import cosima_cookbook as cc
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import xarray as xr
import cartopy.crs as ccrs
import cmocean as cm
from collections import OrderedDict
import cftime
import glob

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client
import matplotlib.path as mpath


import scipy as sp
from xarray.ufuncs import square, sqrt
import scipy.interpolate
import scipy.ndimage
import scipy.io
import scipy.stats
import pandas as pd
from pandas import DataFrame

import statsmodels.api as sm

import xesmf as xe

In [2]:
client = Client(n_workers=2)
client

Client Scheduler: tcp://127.0.0.1:45945 Dashboard: http://127.0.0.1:39453/status,Cluster Workers: 2 Cores: 2 Memory: 5.62 GiB


In [3]:
# Session/experiment:
session = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')
expt='01deg_jra55v140_iaf'
#expt='025deg_jra55_iaf_omip2_cycle6'

In [9]:
# Load grid info:
ht = cc.querying.getvar(expt,'ht',session,n=1).sel(yt_ocean=slice(-90,-55))
ht.load()
hu = cc.querying.getvar(expt,'hu',session,n=1).sel(yu_ocean=slice(-90,-55))
hu.load()
xt_ocean = cc.querying.getvar(expt,'xt_ocean',session, n=-1).fillna(1.0).astype(int) - 1
xt_ocean.load()
yt_ocean = cc.querying.getvar(expt,'yt_ocean',session, n=-1).fillna(1.0).astype(int) - 1
yt_ocean.load()
area_t = cc.querying.getvar(expt,'area_t',session, n=-1).fillna(1.0).astype(int) - 1
area_t.load()

<xarray.DataArray 'area_t' (yt_ocean: 2700, xt_ocean: 3600)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98

In [10]:
# Masks:
tmask = ht.where(~xr.ufuncs.isfinite(ht.where(ht>1000)))
tmask = tmask.where(~xr.ufuncs.isfinite(tmask.where(yt_ocean>-60)))
tmask = ~xr.ufuncs.isnan(tmask).to_dataset(name='tmask')

#wap_mask = ht.where(~xr.ufuncs.isfinite(ht.where( (ht>1000) & (yt_ocean>-60) & (xt_ocean<-75) & (xt_ocean>-63) )))
wap_mask = ht.where(~xr.ufuncs.isfinite(ht.where(ht>1000)))
wap_mask = wap_mask.where(~xr.ufuncs.isfinite(wap_mask.where(yt_ocean>-60)))
wap_mask=wap_mask.where(~xr.ufuncs.isfinite(wap_mask.where(xt_ocean<-75)))
wap_mask=wap_mask.where(~xr.ufuncs.isfinite(wap_mask.where(xt_ocean>-63)))
wap_mask = ~xr.ufuncs.isnan(wap_mask).to_dataset(name='wap_mask')

#bs_mask = ht.where(~xr.ufuncs.isfinite(ht.where((ht>1000) & (yt_ocean>-60) & (xt_ocean<-101) & (xt_ocean>-76) )))
bs_mask = ht.where(~xr.ufuncs.isfinite(ht.where(ht>1000)))
bs_mask = bs_mask.where(~xr.ufuncs.isfinite(bs_mask.where(yt_ocean>-60)))
bs_mask=bs_mask.where(~xr.ufuncs.isfinite(bs_mask.where(xt_ocean<-101)))
bs_mask=bs_mask.where(~xr.ufuncs.isfinite(bs_mask.where(xt_ocean>-76)))
bs_mask = ~xr.ufuncs.isnan(bs_mask).to_dataset(name='bs_mask')

#as_mask = ht.where(~xr.ufuncs.isfinite(ht.where((ht>1000) & (yt_ocean>-60) & (xt_ocean<-127) & (xt_ocean>-102) )))
as_mask = ht.where(~xr.ufuncs.isfinite(ht.where(ht>1000)))
as_mask = as_mask.where(~xr.ufuncs.isfinite(as_mask.where(yt_ocean>-60)))
as_mask=as_mask.where(~xr.ufuncs.isfinite(as_mask.where(xt_ocean<-127)))
as_mask=as_mask.where(~xr.ufuncs.isfinite(as_mask.where(xt_ocean>-102)))
as_mask = ~xr.ufuncs.isnan(as_mask).to_dataset(name='as_mask')

#revised area of AS ice shelves based on Fig. 4e inset of Admusilli https://doi.org/10.1038/s41561-020-0616-z
#as_mask = ht.where(~xr.ufuncs.isfinite(ht.where((ht>1000) & (yt_ocean>-60) & (xt_ocean<-127) & (xt_ocean>-102) )))
as_mask2 = ht.where(~xr.ufuncs.isfinite(ht.where(ht>1000)))
as_mask2 = as_mask2.where(~xr.ufuncs.isfinite(as_mask2.where(yt_ocean>-60)))
as_mask2=as_mask2.where(~xr.ufuncs.isfinite(as_mask2.where(xt_ocean<-137)))
as_mask2=as_mask2.where(~xr.ufuncs.isfinite(as_mask2.where(xt_ocean>-102)))
as_mask2 = ~xr.ufuncs.isnan(as_mask2).to_dataset(name='as_mask2')

#mb_mask = ht.where(~xr.ufuncs.isfinite(ht.where((ht>1000) & (yt_ocean>-60) & (xt_ocean<-158) & (xt_ocean>-103) )))
mb_mask = ht.where(~xr.ufuncs.isfinite(ht.where(ht>1000)))
mb_mask = mb_mask.where(~xr.ufuncs.isfinite(mb_mask.where(yt_ocean>-60)))
mb_mask=mb_mask.where(~xr.ufuncs.isfinite(mb_mask.where(xt_ocean<-158)))
mb_mask=mb_mask.where(~xr.ufuncs.isfinite(mb_mask.where(xt_ocean>-103)))
mb_mask = ~xr.ufuncs.isnan(mb_mask).to_dataset(name='mb_mask')

mb_mask

<xarray.Dataset>
Dimensions:   (xt_ocean: 3600, yt_ocean: 584)
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... -55.12 -55.07 -55.01
Data variables:
    mb_mask   (yt_ocean, xt_ocean) bool False False False ... False False False